In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

In [2]:
csv_file_path = 'taskcsv/wikiHow0.csv'
df = pd.read_csv(csv_file_path)
df.columns = df.columns.str.strip()

In [3]:
data = {
    "input": df["Input"].tolist(),
    "output": df["Output"].tolist()
}

In [4]:
train_inputs, test_inputs, train_outputs, test_outputs = train_test_split(
    data["input"], data["output"], test_size=0.3, random_state=42
)

In [5]:
train_data = {
    "input": train_inputs,
    "output": train_outputs
}
test_data = {
    "input": test_inputs,
    "output": test_outputs
}

In [6]:
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)

In [7]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [11]:
# Freeze all parameters by default
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last 2 decoder layers (layers are in model.model.layers)
for param in model.model.layers[-2:].parameters():  # Unfreeze the last 2 layers
    param.requires_grad = True

# Unfreeze the language modeling head (final output layer)
for param in model.lm_head.parameters():
    param.requires_grad = True


In [12]:
def tokenize_function(examples):
    return tokenizer(examples['input'], examples['output'], truncation=True, padding="max_length", max_length=512)

In [13]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [14]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [15]:
training_args = TrainingArguments(
    output_dir="./results",           # Directory to save results
    evaluation_strategy="epoch",      # Evaluate at the end of each epoch
    learning_rate=5e-5,               # Learning rate for optimizer
    per_device_train_batch_size=8,    # Batch size for training
    per_device_eval_batch_size=8,     # Batch size for evaluation
    num_train_epochs=3,               # Number of training epochs
    weight_decay=0.01,                # Weight decay for regularization
    logging_dir="./logs",             # Directory to store logs
    logging_steps=100,                # Log every 100 steps
    save_steps=1000,                  # Save model every 1000 steps
    save_total_limit=3,               # Keep only the last 3 saved checkpoints
)

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
trainer = Trainer(
    model=model,                      # The pre-trained model to train
    args=training_args,               # Training arguments
    train_dataset=train_dataset,      # Training dataset
    eval_dataset=test_dataset,        # Evaluation dataset
    data_collator=data_collator,      # Data collator
    tokenizer=tokenizer,              # Tokenizer
)

C:\Users\user\AppData\Local\Temp\ipykernel_4396\795119667.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
trainer.train()

  0%|          | 0/1314 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")